In [1]:
import pandas as pd
import geopandas as gpd
import os

# === BASE DIRECTORY ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# === FILE PATHS ===
roads_path = os.path.join(base_dir, "street_network.shp")

# === LOAD SHAPEFILE ===
roads = gpd.read_file(roads_path)

# === CONVERT RELEVANT COLUMNS TO NUMERIC ===
numeric_cols = ['SPEED', 'qDirLanes2', 'qNoAccess', 'qExclude', 'BL_WIDTH', 'BL_REACH', 'PARKALONG', 'StOperNEU', 'ADT_2025', 'FEDERALFUN', 'WIDE_1WAY', 'BL_BLOCK']
roads[numeric_cols] = roads[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [2]:
# === FUNCTION TO CALCULATE LTS_2025 ===
def calculate_LTS_2025(row):
    """Calculate LTS_2025 based on road characteristics."""
    
    protected = row['qProtected']
    no_access = row['qNoAccess']
    exclude = row['qExclude']
    speed = row['SPEED']
    dir_lanes = row['qDirLanes2']
    bl_width = row['BL_WIDTH']
    bl_reach = row['BL_REACH']
    parkalong = row['PARKALONG']
    # illparking = row['ILLPARKING']
    bike_type = row['bike_type2']
    st_oper_neu = row['StOperNEU']
    adt = row['ADT_2025']
    fed_fun = row['FEDERALFUN']
    wide_oneway = row['WIDE_1WAY']
    bl_block = row['BL_BLOCK']  

    lane_infra = ["BL", "BL_LEFT", "BL_MIX", "BL_BUF", "BL_BUF_LEFT", "BL_PK_BUS_BL"]
    sep_infra = ["SUP", "SUP_NAT", "SUP_MINOR", "SBL", "SBL_LEFT", "SBL_BL", "SBL_MIX", "CARFREE"]
    mix_infra = ["MIX_CONTRA", "MIX_SCONTRA", "BUS_BL", "BUS_BL_LEFT", "SLM", "SLMTC"]


    if no_access in (1, 98, 99):
        return 6  # freeway (ramps)
    elif exclude in (1, 5) or (bike_type and bike_type in ("WALK_YR_BIKE")):
        return 5  # cemetary, private property, peds only, etc.
    elif fed_fun == 0:
        return 1  # Parks, alleys, bike paths, streets with almost no traffic
    elif bike_type in sep_infra:
        return 1  # Separated from traffic
    elif protected == 1:
        return 1 # Separated from traffic
    

    elif ((bl_width >= 4) or bike_type in lane_infra) and (parkalong == 0):
        # Conventional bike lanes not adjacent to parking
        if (
            (dir_lanes >= 3 and speed > 38.5) or 
            (dir_lanes == 2 and speed > 43.5 and bl_width < 6) or
            (dir_lanes <= 1 and speed > 48.5 and bl_width < 6)
        ):
            return 4
        elif (dir_lanes >= 3) or (speed > 38.5):
            return 3
        elif (
            (dir_lanes == 2) or
            (speed > 33.5) or
            (dir_lanes <= 1 and bl_width < 6) or
            (dir_lanes <= 1 and speed > 28.5)
        ):
            return 2
        elif (dir_lanes <= 1 and bl_width >= 6):
            return 1
        else:
            return 98 # For other cases that do not meet conditions


    elif (bl_width >= 4 or bike_type in lane_infra) and (parkalong == 1): # and (bl_reach >= 12):
        # Conventional bike lanes adjacent to parking
        if (dir_lanes <= 1 and speed <= 28.5 and bl_reach >= 15):
            return 1
        elif (
            (dir_lanes <= 1 and speed <= 38.5 and bl_reach >= 15) or
            (dir_lanes <= 1 and speed <= 33.5 and bl_reach < 15) or
            (dir_lanes >= 2 and st_oper_neu == 1 and speed <= 28.5 and bl_reach >= 15) or
            (dir_lanes == 2 and speed <= 28.5 and bl_reach >= 15)
        ):
            return 2
        else:
            return 3


    elif (
        (bl_width < 4) or 
        ((bl_width >= 4) and (bl_block == 1)) or 
        ((bl_width >= 4) and (parkalong > 0) and (bl_reach < 12)) or
        bike_type in mix_infra
    ):
        # Mixed traffic conditions
        if (
            # 3+ thru lanes per direction
            (dir_lanes >= 3 and speed > 28.5) or

            # 2 thru lanes per direction
            (dir_lanes == 2 and adt > 8_000 and speed > 28.5) or 
            (dir_lanes == 2 and adt <= 8_000 and speed > 38.5) or

            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 1_000 and speed > 38.5) or
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 600 and speed > 43.5) or

            # Two-way 1+1 per direction with CL or wide one-way street
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_500 and speed > 38.5) or
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_000 and speed > 43.5) or

            # Unlaned two-way streets
            (dir_lanes == 0 and adt > 3_000 and speed > 38.5) or
            (dir_lanes == 0 and adt > 1_500 and speed > 43.5)
        ):
            return 4

        elif (
            # 2+ thru lanes per direction
            dir_lanes >= 2 or

            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 1_000 and speed > 23.5) or
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 600 and speed > 33.5) or
            (dir_lanes <= 1 and st_oper_neu == 1 and speed > 38.5) or

            # Two-way 1+1 per direction with CL or wide one-way street
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_500 and speed > 23.5) or
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_000 and speed > 33.5) or
            ((dir_lanes == 1 or wide_oneway == 1) and speed > 38.5) or

            # Unlaned two-way streets
            (dir_lanes == 0 and adt > 3_000 and speed > 28.5) or
            (dir_lanes == 0 and adt > 750 and speed > 33.5) or
            (dir_lanes == 0 and speed > 38.5)
        ):
            return 3

        elif (
            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1 and adt > 600) or
            (dir_lanes <= 1 and st_oper_neu == 1 and speed > 28.5) or

            # Two-way 1+1 per direction with CL or wide one-way street
            ((dir_lanes == 1 or wide_oneway == 1) and adt > 1_000) or
            ((dir_lanes == 1 or wide_oneway == 1) and speed > 28.5) or

            # Unlaned two-way streets
            (dir_lanes == 0 and adt > 1_500) or
            (dir_lanes == 0 and speed > 28.5)
        ):
            return 2

        elif (
            # Narrow one-way single-lane street
            (dir_lanes <= 1 and st_oper_neu == 1) or

            # Two-way 1+1 per direction with CL or wide one-way street
            (dir_lanes == 1 or wide_oneway == 1) or

            # Unlaned two-way streets
            dir_lanes == 0
        ):
            return 1
        else:
            return 99  # For other cases that do not meet conditions

    else:
        return 96  # Default return for other cases

# === CALCULATE LTS_2025 FOR ROADS ===
roads['LTS_2025'] = roads.apply(calculate_LTS_2025, axis=1)

# === SAVE THE UPDATED ROADS SHAPEFILE ===
roads.to_file(roads_path, driver="ESRI Shapefile")
 
# Output summary
print("LTS_2025 calculation complete.")
print(roads['LTS_2025'].value_counts(dropna=False))

LTS_2025 calculation complete.
LTS_2025
1    18520
5     6278
3     5485
2     3363
6     1885
4     1265
Name: count, dtype: int64


In [3]:
roads['LTS_2018'].value_counts(dropna=False)

LTS_2018
1    17648
3     7140
5     6055
2     2186
6     1839
4     1682
0      246
Name: count, dtype: int64

In [4]:
# Assuming roads is your GeoDataFrame
lts_columns = [col for col in roads.columns if "LTS" in col]
print(lts_columns)

['qLTS', 'qLTS_Retn', 'qLTS_Own', 'LTS2006', 'LTS2014', 'LTSEmerald', 'LTSBForE', 'LTS_DT_Imp', 'LTS_DT_BE', 'LTS_DT_Col', 'LTS_Colum', 'LTS_All_Im', 'LTS_2017', 'LTS_2018', 'LTS_2025', 'LTS_2025b', 'LTS_DIFF']


In [5]:
# Filter rows with valid LTS values
valid_values = {1, 2, 3, 4}
filtered_rows = roads[roads["LTS_2018"].isin(valid_values) & roads["LTS_2025"].isin(valid_values)]

# Count instances for different conditions
cases_2018_gt_2025 = (filtered_rows["LTS_2018"] > filtered_rows["LTS_2025"]).sum()
cases_2018_eq_2025 = (filtered_rows["LTS_2018"] == filtered_rows["LTS_2025"]).sum()
cases_2018_lt_2025 = (filtered_rows["LTS_2018"] < filtered_rows["LTS_2025"]).sum()

# Calculate transitions
breakdown = {}
for lts_2025, lts_2018 in [(1, 1), (2, 2), (3, 3), (4, 4), 
                           (1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), 
                           (3, 1), (3, 2), (3, 4), (4, 1), (4, 2), (4, 3)]:
    count = ((filtered_rows["LTS_2018"] == lts_2018) & (filtered_rows["LTS_2025"] == lts_2025)).sum()
    breakdown[(lts_2025, lts_2018)] = count

# Print general summary
print("\nGeneral Summary:")
print(f"Cases where Theja > NatDave: {cases_2018_gt_2025}")
print(f"Cases where Theja = NatDave: {cases_2018_eq_2025}")
print(f"Cases where Theja < NatDave: {cases_2018_lt_2025}")

# Print results
print("\nDetailed Breakdown:")

if cases_2018_gt_2025:
    print("\nCases where Theja > NatDave:")
    for (lts_2025, lts_2018), count in breakdown.items():
        if lts_2018 > lts_2025 and count > 0:
            print(f"Theja says {lts_2018} but NatDave says {lts_2025}: {count}")

if cases_2018_eq_2025:
    print("\nCases where Theja = NatDave:")
    for (lts_2025, lts_2018), count in breakdown.items():
        if lts_2018 == lts_2025 and count > 0:
            print(f"Theja says {lts_2018} but NatDave says {lts_2025}: {count}")

if cases_2018_lt_2025:
    print("\nCases where Theja < NatDave:")
    for (lts_2025, lts_2018), count in breakdown.items():
        if lts_2018 < lts_2025 and count > 0:
            print(f"Theja says {lts_2018} but NatDave says {lts_2025}: {count}")


General Summary:
Cases where Theja > NatDave: 2755
Cases where Theja = NatDave: 25300
Cases where Theja < NatDave: 532

Detailed Breakdown:

Cases where Theja > NatDave:
Theja says 2 but NatDave says 1: 297
Theja says 3 but NatDave says 1: 633
Theja says 4 but NatDave says 1: 243
Theja says 3 but NatDave says 2: 1261
Theja says 4 but NatDave says 2: 82
Theja says 4 but NatDave says 3: 239

Cases where Theja = NatDave:
Theja says 1 but NatDave says 1: 17303
Theja says 2 but NatDave says 2: 1784
Theja says 3 but NatDave says 3: 5095
Theja says 4 but NatDave says 4: 1118

Cases where Theja < NatDave:
Theja says 1 but NatDave says 2: 236
Theja says 1 but NatDave says 3: 44
Theja says 2 but NatDave says 3: 105
Theja says 3 but NatDave says 4: 147


In [6]:
# Check where the values are not the same
diff_mask = filtered_rows["LTS_2018"] != filtered_rows["LTS_2025"]
# Count the number of instances where the values differ
num_diff = diff_mask.sum()

# Calculate the proportion of differing values
proportion_diff = num_diff / len(roads)

# Print results
print(f"Number of differing rows: {num_diff} out of {len(roads)}")
print(f"Proportion of differing rows: {proportion_diff:.2%}")

Number of differing rows: 3287 out of 36796
Proportion of differing rows: 8.93%


In [7]:
# Calculate road lengths in meters
roads["LENGTH_METERS"] = roads.geometry.length

# Convert lengths to miles
roads["LENGTH_MILES"] = roads["LENGTH_METERS"] * 0.000621371

# Calculate total miles for each LTS_2025 value
lts_miles = roads.groupby("LTS_2025")["LENGTH_MILES"].sum()

# Print the results
print("Total miles for each LTS_2025 value:")
print(lts_miles)

Total miles for each LTS_2025 value:
LTS_2025
1    1024.516950
2     165.565144
3     263.471136
4      72.191847
5     316.075461
6     124.071922
Name: LENGTH_MILES, dtype: float64


In [8]:
roads["LENGTH_MILES"].sum()

1965.8924581837405

In [9]:
# Filter rows where LTS_2025 < LTS_2018
improve = roads[roads["LTS_2025"] < roads["LTS_2018"]]

# Calculate the total length in miles
total_length_miles = improve["LENGTH_MILES"].sum()

# Print the result
print(f"Total length in miles where LTS_2025 < LTS_2018: {total_length_miles}")

Total length in miles where LTS_2025 < LTS_2018: 126.1962324304479


In [10]:
roads['SPEED'].value_counts(dropna=False)

SPEED
30    19483
25    15352
35     1176
40      285
0       246
20      235
15       15
27        4
Name: count, dtype: int64